In [3]:
# Import required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error, r2_score, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

# from google.colab import files
# uploaded = files.upload()
from google.colab import drive
drive.mount('/content/drive')

# Load the dataset
data = pd.read_csv('/content/drive/My Drive/[[[[ECCE 2025 CUET]]]]/Implementation of Existing Model/online_shoppers_intention.csv')

# Define numerical and categorical columns
numerical_columns = ['Administrative', 'Administrative_Duration', 'Informational', 'Informational_Duration',
                     'ProductRelated', 'ProductRelated_Duration', 'BounceRates', 'ExitRates', 'PageValues', 'SpecialDay']
categorical_columns = ['Month', 'OperatingSystems', 'Browser', 'Region', 'TrafficType', 'VisitorType', 'Weekend']

# Separate features and target variable
X = data[numerical_columns + categorical_columns]
y = data['Revenue']  # Assuming 'Revenue' is the dependent variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Preprocess the data**

In [4]:
# Preprocess the data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(drop='first'), categorical_columns)
    ]
)

X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

**Train the Linear Regression model and make prediction**

In [6]:
# Step 4: Train the Decision Tree classifier
dt_model = DecisionTreeClassifier(random_state=42)
# Fit the model using the transformed data (X_train_transformed)
dt_model.fit(X_train_transformed, y_train)

# Step 5: Evaluate the model
# Predict using the transformed data (X_test_transformed)
y_pred = dt_model.predict(X_test_transformed)

**Evaluate the model**

In [7]:
# Calculate accuracy and print evaluation metrics
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")
print("Classification Report:")
print(classification_report(y_test, y_pred))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

Accuracy: 0.85
Classification Report:
              precision    recall  f1-score   support

       False       0.91      0.90      0.91      2055
        True       0.54      0.57      0.56       411

    accuracy                           0.85      2466
   macro avg       0.73      0.74      0.73      2466
weighted avg       0.85      0.85      0.85      2466

Confusion Matrix:
[[1857  198]
 [ 175  236]]


**Feature Importance (Coefficients)**

In [9]:
# Feature Importance (Coefficients)
feature_names = (
    numerical_columns +
    preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_columns).tolist()
)


# Decision Trees don't have coefficients like linear models.
# We'll use feature_importances_ instead.
feature_importances = dt_model.feature_importances_  # Get feature importances from the trained Decision Tree

coefficients = pd.DataFrame({
    'Feature': feature_names,
    'Importance': feature_importances  # Use 'Importance' instead of 'Coefficient'
}).sort_values(by='Importance', ascending=False)  # Sort by importance

print("\nFeature Importance:")
print(coefficients)


Feature Importance:
                    Feature  Importance
8                PageValues    0.417832
5   ProductRelated_Duration    0.094117
7                 ExitRates    0.071235
6               BounceRates    0.069583
1   Administrative_Duration    0.056267
..                      ...         ...
62           TrafficType_18    0.000000
63           TrafficType_19    0.000000
35               Browser_11    0.000000
22       OperatingSystems_5    0.000000
56           TrafficType_12    0.000000

[68 rows x 2 columns]


In [11]:
# Make predictions and convert to binary using a threshold (e.g., 0.5)
y_pred = dt_model.predict(X_test_transformed) # Changed 'model' to 'dt_model'
y_pred_binary = np.where(y_pred > 0.5, 1, 0)  # Convert to 0 or 1 based on threshold

# Calculate classification metrics using the binary predictions
accuracy = accuracy_score(y_test, y_pred_binary)
precision = precision_score(y_test, y_pred_binary)
recall = recall_score(y_test, y_pred_binary)
f1 = f1_score(y_test, y_pred_binary)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

Accuracy: 0.85
Precision: 0.54
Recall: 0.57
F1 Score: 0.56
